In [33]:
import requests
import pandas as pd
import numpy as np
import json

In [ ]:
# check API connection status

url = "https://besttime.app/api/v1/keys/pri_213fe6a21137499da100e7bf8b3553f5"

response = requests.request("GET", url)

print(response.json())

In [ ]:
# all clubs in singapore                                                                                                                                     q# all clubs in singapore

url = "https://besttime.app/api/v1/venues/search"

params = {
    'api_key_private': 'pri_213fe6a21137499da100e7bf8b3553f5',
    'q': 'all bars and nightclubs in Singapore',
    'num': 200,
    'fast': False,
    'foot_traffic': 'limited',
    'hour_min': 18,
    'hour_max': 0,
    'day_int': 5,
    'types': ['BEER','BREWERY','WINERY','CLUBS']
}

response = requests.request("POST", url, params=params)
response_json = response.json()

In [ ]:
# all bars in singapore

url = "https://besttime.app/api/v1/venues/search"

params = {
    'api_key_private': 'pri_213fe6a21137499da100e7bf8b3553f5',
    'q': 'all bars in Singapore',
    'format'
    'num': 200,
    'fast': False,
    'foot_traffic': 'limited',
    'hour_min': 18,
    'hour_max': 0,
    'day_int': 5,
    'types': ['BAR']
}

response = requests.request("POST", url, params=params)
print(response.json())

In [ ]:
# all fast food restaurants singapore

url = "https://besttime.app/api/v1/venues/search"

params = {
    'api_key_private': 'pri_213fe6a21137499da100e7bf8b3553f5',
    'q': 'all fast food restaurants in Singapore',
    'num': 200,
    'fast': False,
    'types': ['FAST_FOOD']
}

response = requests.request("POST", url, params=params)
print(response.json())

In [34]:
# commit all grab restaurants open past midnight to the same collection
restaurants_df = pd.read_csv("dataset/midnight_restaurants.csv")

In [38]:
# Function to get crowd density data from besttime using restaurant name and address
def get_cd_json(name, address, is_midnight_restaurant):
    if is_midnight_restaurant == False:
        return np.nan
    
    url = "https://besttime.app/api/v1/forecasts"
    params = {
        'api_key_private': 'pri_213fe6a21137499da100e7bf8b3553f5',
        'venue_name': name,
        'venue_address': address
    }

    response = requests.request("POST", url, params=params)
    return response.json()

In [39]:
restaurants_df['cd_json'] = restaurants_df.apply(lambda x: get_cd_json(x['name'], x['address'], x['midnight_restaurant']), axis=1)

In [40]:
# Initialize empty columns to store Friday and Saturday crowd density data
restaurants_df['fri_cd'], restaurants_df['sat_cd'] = np.nan, np.nan

In [75]:
def get_fri_sat_cd(row):
    fri_cd, sat_cd = np.nan, np.nan
    cd_json = row['cd_json']
    if type(cd_json) != dict:
        return np.nan, np.nan

    if cd_json['status'] == 'OK':
        analysis = cd_json['analysis']
        for day in analysis: 
            day_text = day['day_info']['day_text']
            day_raw = day['day_raw']
            if day_text == 'Friday':
                fri_cd = day_raw
            if day_text == 'Saturday':
                sat_cd = day_raw
    return fri_cd, sat_cd

In [80]:
restaurants_df[['fri_cd', 'sat_cd']] = restaurants_df.apply(lambda x: get_fri_sat_cd(x), axis=1, result_type='expand')

In [83]:
restaurants_df.to_csv("dataset/besttime/restaurants.csv")